# Matrix

### Purpose
- Demonstrate graph connecting output to input. The example shows connection between cities in a trip. 
- Build a dependency matrix

In [23]:
import logging
logging.basicConfig()
logger = logging.getLogger('dependencynet')
logger.setLevel(logging.WARN)

In [24]:
# clarification for linter
from IPython.display import display

In [25]:
# remove these lines if you use the pypi package
import sys
sys.path.append("../../..") # go to parent dir

In [26]:
from dependencynet.schema import SchemaBuilder
from dependencynet.model import ModelBuilder
from dependencynet.network.graphbuilder import GraphBuilder, LevelNode, InputNode, OutputNode
from dependencynet.network.stylebuilder import StyleBuilder
from dependencynet.network.graphviewer import GraphViewer
from dependencynet.network.graphml import GraphMLConverter

## Loads and prepare data

Check the notebook for more information on the dataset and the steps requiored to prepare the graph.

In [27]:
%run example-graphml-trips.ipynb

,area,country,town,flight_in,flight_out
0,Europe,France,Paris,fl2,fl3
1,Europe,France,Lyon,NaN,fl1
2,Europe,UK,London,fl4,fl5
3,Europe,Italia,Rome,fl1,fl2
4,Asia,Japan,Tokyo,fl3,fl4


CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeSpacing': 10, 'edgeLenghVal': 10, 'rankDir': 'LR'}, cy…

Done


## Matrix

In [28]:
graph_model.remove_category('area')
graph_model.remove_category('country')
graph_model.merge_connection('flight_out', 'flight_in', 'flight')
graph_model.fold_category('flight')

In [29]:
graph_model.pretty_print()

['Nodes:',
 'town level - A01C01T01 Paris',
 'town level - A01C01T02 Lyon',
 'town level - A01C02T01 London',
 'town level - A01C03T01 Rome',
 'town level - A02C01T01 Tokyo',
 '',
 'Edges:',
 'town level A01C01T01 -> town level A02C01T01',
 'town level A01C01T02 -> town level A01C03T01',
 'town level A01C03T01 -> town level A01C01T01',
 'town level A02C01T01 -> town level A01C02T01']

In [44]:
def to_checkmark(v):
    if not isinstance(v, str):
        if v > 0:
            v = 'X'
        else:
            v = ''
    return v

#### Matrix 1

In [30]:
import networkx as nx
#nx.to_numpy_matrix(graph_model.G, nodelist=[0,1,2])
matrix = nx.to_numpy_matrix(graph_model.G)
display(matrix)

matrix([[0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [47]:
towns = source_df['town'].tolist()

In [49]:
matrix_df = pd.DataFrame(data=matrix, index=towns, columns=towns)

In [50]:
checkmarks_df = matrix_df.applymap(to_checkmark)

In [51]:
display(checkmarks_df)

,Paris,Lyon,London,Rome,Tokyo
Paris,,,,,X
Lyon,,,,X,
London,,,,,
Rome,X,,,,
Tokyo,,,X,,


#### Matrix 2

In [40]:
data_df = source_df.copy()
no_flight_in_mask = pd.isnull(data_df['flight_in'])
clean_df = data_df[~no_flight_in_mask]
one_hot_df = pd.get_dummies(clean_df['flight_in'], prefix='', prefix_sep='')
extended_df = pd.concat([clean_df, one_hot_df], axis=1)
key = ['area', 'country', 'town']
matrix_df = extended_df.groupby(by=key).sum()

In [41]:
display(matrix_df)

fl1  fl2  fl3  fl4
area   country town                      
Asia   Japan   Tokyo     0    0    1    0
Europe France  Paris     0    1    0    0
       Italia  Rome      1    0    0    0
       UK      London    0    0    0    1

In [45]:
checkmarks_df = matrix_df.applymap(to_checkmark)

In [46]:
display(checkmarks_df)

fl1 fl2 fl3 fl4
area   country town                  
Asia   Japan   Tokyo            X    
Europe France  Paris        X        
       Italia  Rome     X            
       UK      London               X